# Dependencies loading

In [147]:
#libraries
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import pickle
from joblib import dump, load

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

np.random.seed(2024)
pd.set_option("display.max_columns", 500)

## Data loading

In [148]:
FE_data_train = pd.read_csv('../input_data/fe_X_train.csv',index_col=0)
FE_y_train = pd.read_csv('../input_data/fe_y_train.csv',index_col=0)
RE_data_train = pd.read_csv('../input_data/re_X_train.csv',index_col=0)
RE_y_train = pd.read_csv('../input_data/re_y_train.csv',index_col=0)
fe_results = pd.read_csv('../input_data/fe_results.csv',index_col=0)

In [149]:
FE_data_train

,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Education_Level_NA,Income_Category_NA,Gender_M,MS_Married,MS_Single,ln_Total_Trans_Amt,ln_Credit_Limit,ln_Avg_Transaction,Avg_Transaction,Credit_utilization
0,27.0,0.0,3.0,2.0,0.0,36.0,5.0,1.0,0.0,13439.0,2003.0,11436.0,0.936,4715.0,86.0,0.720,0.149,0.0,1.0,True,True,False,8.458504,9.505916,4.004157,54.825581,0.149044
1,46.0,0.0,0.0,2.0,1.0,36.0,6.0,3.0,2.0,30501.0,2030.0,28471.0,0.560,2525.0,45.0,0.500,0.067,0.0,0.0,False,False,True,7.833996,10.325515,4.027334,56.111111,0.066555
2,46.0,2.0,1.0,0.0,0.0,37.0,3.0,1.0,1.0,6118.0,1092.0,5026.0,0.757,4434.0,81.0,0.620,0.178,0.0,0.0,True,True,False,8.397057,8.718991,4.002608,54.740741,0.178490
3,57.0,2.0,3.0,0.0,0.0,45.0,6.0,2.0,2.0,1438.3,740.0,698.3,0.698,4970.0,87.0,0.933,0.514,0.0,0.0,False,True,False,8.511175,7.271217,4.045267,57.126437,0.514496
4,56.0,1.0,2.0,3.0,0.0,36.0,3.0,3.0,3.0,34516.0,1477.0,33039.0,1.234,2395.0,50.0,0.724,0.043,0.0,0.0,True,True,False,7.781139,10.449178,3.869116,47.900000,0.042792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8096,65.0,1.0,0.0,0.0,0.0,55.0,3.0,2.0,4.0,8963.0,0.0,8963.0,0.744,4511.0,76.0,0.689,0.000,0.0,0.0,False,False,False,8.414274,9.100860,4.083541,59.355263,0.000000
8097,40.0,1.0,1.0,0.0,0.0,30.0,6.0,2.0,1.0,2579.0,1959.0,620.0,0.618,4300.0,69.0,0.683,0.760,0.0,0.0,False,True,False,8.366370,7.855157,4.132264,62.318841,0.759597
8098,52.0,2.0,4.0,3.0,1.0,48.0,6.0,6.0,1.0,34516.0,1473.0,33043.0,0.642,4254.0,73.0,0.587,0.043,1.0,0.0,False,False,False,8.355615,10.449178,4.065156,58.273973,0.042676
8099,44.0,4.0,3.0,1.0,0.0,40.0,6.0,5.0,3.0,1572.0,874.0,698.0,0.947,4996.0,80.0,0.739,0.556,0.0,0.0,False,True,False,8.516393,7.360104,4.134366,62.450000,0.555980


In [150]:
FE_y_train

,Attrition_Flag
6150,0
4368,0
4175,0
7352,0
1285,0
...,...
4753,0
5370,0
4621,0
6774,0


In [151]:
RE_data_train

,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Education_Level_NA,Income_Category_NA,Gender_M,MS_Married,MS_Single,ln_Total_Trans_Amt,ln_Credit_Limit,ln_Avg_Transaction,Avg_Transaction,Credit_utilization
0,27.000000,0.000000,3.000000,2.000000,0.00000,36.000000,5.000000,1.000000,0.000000,13439.000000,2003.000000,11436.000000,0.936000,4715.000000,86.000000,0.720000,0.149000,0.0,1.000000,True,True,False,8.458504,9.505916,4.004157,54.825581,0.149044
1,46.000000,0.000000,0.000000,2.000000,1.00000,36.000000,6.000000,3.000000,2.000000,30501.000000,2030.000000,28471.000000,0.560000,2525.000000,45.000000,0.500000,0.067000,0.0,0.000000,False,False,True,7.833996,10.325515,4.027334,56.111111,0.066555
2,46.000000,2.000000,1.000000,0.000000,0.00000,37.000000,3.000000,1.000000,1.000000,6118.000000,1092.000000,5026.000000,0.757000,4434.000000,81.000000,0.620000,0.178000,0.0,0.000000,True,True,False,8.397057,8.718991,4.002608,54.740741,0.178490
3,57.000000,2.000000,3.000000,0.000000,0.00000,45.000000,6.000000,2.000000,2.000000,1438.300000,740.000000,698.300000,0.698000,4970.000000,87.000000,0.933000,0.514000,0.0,0.000000,False,True,False,8.511175,7.271217,4.045267,57.126437,0.514496
4,56.000000,1.000000,2.000000,3.000000,0.00000,36.000000,3.000000,3.000000,3.000000,34516.000000,1477.000000,33039.000000,1.234000,2395.000000,50.000000,0.724000,0.043000,0.0,0.000000,True,True,False,7.781139,10.449178,3.869116,47.900000,0.042792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13593,43.412261,2.396935,1.603065,3.301533,0.00000,33.206130,2.507663,2.000000,4.000000,26377.957089,0.000000,26377.957089,0.567925,2095.393487,45.095402,0.627923,0.000000,0.0,0.698467,False,True,False,7.647408,10.180247,3.839102,46.529803,0.000000
13594,43.051042,4.978125,0.000000,0.000000,0.00000,36.036459,4.978125,3.000000,3.985416,2689.452089,2383.044779,306.407310,0.365963,1942.217724,52.905207,0.654724,0.886504,0.0,0.007292,False,True,True,7.571560,7.897091,3.603320,36.739874,0.886082
13595,46.616168,2.097795,0.601470,1.000000,0.00000,42.014698,2.699265,3.202940,2.000000,3631.537590,334.248724,3297.288866,0.558371,2069.645563,33.706614,0.664179,0.090205,0.0,0.699265,True,False,True,7.635039,8.197015,4.124397,62.150603,0.090386
13596,39.154870,2.309739,0.000000,3.845130,0.84513,21.632782,2.619478,1.309739,2.000000,34516.000000,761.309228,33754.690772,0.873368,7866.267608,75.929217,0.730497,0.022071,0.0,0.000000,False,False,False,8.969456,10.449178,4.640049,103.559109,0.022057


In [152]:
RE_y_train

,Attrition_Flag
0,0
1,0
2,0
3,0
4,0
...,...
13593,1
13594,1
13595,1
13596,1


In [153]:
fe_results[~fe_results['Variable'].isin(["Avg_Transaction","Credit_Limit","Avg_Utilization_Ratio","Total_Trans_Amt"])]

,Variable,Variance,Chi2,p_Chi2,F_S,M_I,EN_coef,selector
0,Customer_Age,64.11,2.705,0.100,1.958,0.000,-1.480425e-04,0.000000
1,Dependent_count,1.69,2.431,0.119,3.373,0.000,1.058961e-02,0.035704
2,Education_Level,1.83,5.758,0.016,6.524,0.000,3.233110e-03,0.001916
3,Income_Category,1.78,0.985,0.321,0.757,0.000,-5.498340e-03,-0.024531
4,Card_Category,0.11,0.004,0.949,0.003,0.004,2.151725e-02,0.000000
5,Months_on_book,63.90,1.552,0.213,0.874,0.003,-2.697242e-04,-0.001222
6,Total_Relationship_Count,2.43,107.034,0.000,171.491,0.011,-3.907358e-02,-0.128604
7,Months_Inactive_12_mon,1.02,83.554,0.000,195.501,0.019,3.848824e-02,0.124742
8,Contacts_Count_12_mon,1.23,180.411,0.000,377.836,0.029,4.327946e-02,0.143296
10,Total_Revolving_Bal,665432.24,318001.885,0.000,597.719,0.085,-6.695211e-05,-0.000282


In [154]:
fe_columns = fe_results[~fe_results['Variable'].isin(["Avg_Transaction","Credit_Limit","Avg_Utilization_Ratio","Total_Trans_Amt"])].Variable.to_list()

In [155]:
RE_data_train[fe_columns]

,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Education_Level_NA,Income_Category_NA,Gender_M,MS_Married,MS_Single,ln_Total_Trans_Amt,ln_Credit_Limit,ln_Avg_Transaction,Credit_utilization
0,27.000000,0.000000,3.000000,2.000000,0.00000,36.000000,5.000000,1.000000,0.000000,2003.000000,11436.000000,0.936000,86.000000,0.720000,0.0,1.000000,True,True,False,8.458504,9.505916,4.004157,0.149044
1,46.000000,0.000000,0.000000,2.000000,1.00000,36.000000,6.000000,3.000000,2.000000,2030.000000,28471.000000,0.560000,45.000000,0.500000,0.0,0.000000,False,False,True,7.833996,10.325515,4.027334,0.066555
2,46.000000,2.000000,1.000000,0.000000,0.00000,37.000000,3.000000,1.000000,1.000000,1092.000000,5026.000000,0.757000,81.000000,0.620000,0.0,0.000000,True,True,False,8.397057,8.718991,4.002608,0.178490
3,57.000000,2.000000,3.000000,0.000000,0.00000,45.000000,6.000000,2.000000,2.000000,740.000000,698.300000,0.698000,87.000000,0.933000,0.0,0.000000,False,True,False,8.511175,7.271217,4.045267,0.514496
4,56.000000,1.000000,2.000000,3.000000,0.00000,36.000000,3.000000,3.000000,3.000000,1477.000000,33039.000000,1.234000,50.000000,0.724000,0.0,0.000000,True,True,False,7.781139,10.449178,3.869116,0.042792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13593,43.412261,2.396935,1.603065,3.301533,0.00000,33.206130,2.507663,2.000000,4.000000,0.000000,26377.957089,0.567925,45.095402,0.627923,0.0,0.698467,False,True,False,7.647408,10.180247,3.839102,0.000000
13594,43.051042,4.978125,0.000000,0.000000,0.00000,36.036459,4.978125,3.000000,3.985416,2383.044779,306.407310,0.365963,52.905207,0.654724,0.0,0.007292,False,True,True,7.571560,7.897091,3.603320,0.886082
13595,46.616168,2.097795,0.601470,1.000000,0.00000,42.014698,2.699265,3.202940,2.000000,334.248724,3297.288866,0.558371,33.706614,0.664179,0.0,0.699265,True,False,True,7.635039,8.197015,4.124397,0.090386
13596,39.154870,2.309739,0.000000,3.845130,0.84513,21.632782,2.619478,1.309739,2.000000,761.309228,33754.690772,0.873368,75.929217,0.730497,0.0,0.000000,False,False,False,8.969456,10.449178,4.640049,0.022057


In [156]:
data = [[FE_data_train,FE_y_train],[RE_data_train,RE_y_train],[RE_data_train[fe_columns],RE_y_train]]

# Models 

For our analysis we will create 4 models - Logistic regression as baseline model, KNN, SVM and Random Forest.

For each models we will create 3 models using stratified k-fold cross validation:
- models with every variable
- models with variables from feature selection 
- models with variables from feature selection and balanced target variable

## Cross validation and scoring

In [47]:
scoring = ('roc_auc','matthews_corrcoef','f1')
cv = StratifiedKFold(n_splits=8, random_state = 2024, shuffle = True)

## Logistic regression

In [48]:
LR = LogisticRegression(max_iter=5000)

In [49]:
res = []
for i in range(len(data)):
    res.append(cross_validate(LR, data[i][0].values, data[i][1].values.ravel(), scoring = scoring ,cv=cv))

### Results 

In [50]:
pd.DataFrame([pd.DataFrame(res[0]).mean(),pd.DataFrame(res[1]).mean(),pd.DataFrame(res[2]).mean()]).T.rename(columns={0:'Base',1:'Rebalanced',2:'Rebalanced_Selected'})

,Base,Rebalanced,Rebalanced_Selected
fit_time,0.839712,2.961081,1.713952
score_time,0.006386,0.008220,0.006585
test_roc_auc,0.922999,0.951438,0.946762
test_matthews_corrcoef,0.611727,0.766805,0.741568
test_f1,0.658675,0.883843,0.871614


Based on every score rebalanced dataset using SMOTE with every feature performs the best

test_roc_auc (0.951438):

-ROC-AUC (Receiver Operating Characteristic - Area Under the Curve) is a metric that evaluates the ability of the model to discriminate between positive and negative examples. An ROC-AUC of 0.951438 indicates a strong performance, with a value closer to 1 suggesting better discrimination. But we also have to remember that ROC curve is not the best for our imbalanced dataset.

test_matthews_corrcoef (0.611727):

-The Matthews Correlation Coefficient (MCC) is a measure of the quality of binary classifications. An MCC of 0.766805 suggests a moderate level of correlation between the predicted and actual classifications. A value of 1 indicates a perfect prediction, 0 indicates no better than random, and -1 indicates total disagreement.

test_f1 (0.883843):

F1 score is the harmonic mean of precision and recall. An F1 score of 0.883843 indicates a reasonably balanced trade-off between precision and recall. It ranges from 0 to 1, where 1 is the best possible F1 score.

In summary, the model seems to perform well with a high ROC-AUC, a moderate Matthews Correlation Coefficient, and a decent F1 score. Additionally, the fit and prediction times appear to be reasonable, considering the context of the task.

### Hyperparametru tuning

The only hyperparameter in logistic regression is C - regularization term and regularization type

In [52]:
param_grid = {
    'C': 10**np.linspace(0,3,5),
    'penalty' : ['l1', 'l2', 'elasticnet']
    }

In [53]:
grid_result = GridSearchCV(LR, param_grid, scoring = "f1", cv = cv, n_jobs = -1).fit(RE_data_train.values,RE_y_train.values.ravel())

/Users/anfrejter/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/anfrejter/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

In [54]:
grid_result.best_params_

{'C': 1.0, 'penalty': 'l2'}

This result corresponds with the best hyperparameters found by a grid search. In this case, the best value found is 1.0, suggesting a moderate level of regularization. l2' refers to L2 regularization, also known as Ridge regularization

In [55]:
grid_result.best_score_

0.8838427517542184

It turns out that our model with default parameters is the best

In [56]:
LR = LogisticRegression(penalty='l2', C = 1, max_iter=5000)

In [57]:
LR.fit(RE_data_train,RE_y_train)

/Users/anfrejter/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1, max_iter=5000)

In [58]:
dump(LR, '../models/Logistic_Regression.joblib') 

['../models/Logistic_Regression.joblib']

## KNN

### Feature engineering 

Before modeling we have to normalize our data in order for KNN to work properly.

In [22]:
scaler = StandardScaler()
scal_FE_data_train = pd.DataFrame(scaler.fit_transform(FE_data_train),columns=FE_data_train.columns)
scaler = StandardScaler()
scal_RE_data_train = pd.DataFrame(scaler.fit_transform(RE_data_train),columns=FE_data_train.columns)

In [23]:
scal_data = [[scal_FE_data_train,FE_y_train],[scal_RE_data_train,RE_y_train],[scal_RE_data_train[fe_columns],RE_y_train]]

In [24]:
KNN = KNeighborsClassifier(n_neighbors=5)

In [25]:
res = []
for i in range(len(scal_data)):
    res.append(cross_validate(KNN, scal_data[i][0].values, scal_data[i][1].values.ravel(), scoring = scoring ,cv=cv))

### Results

In [26]:
pd.DataFrame([pd.DataFrame(res[0]).mean(),pd.DataFrame(res[1]).mean(),pd.DataFrame(res[2]).mean()]).T.rename(columns={0:'Base',1:'Rebalanced',2:'Rebalanced_Selected'})

,Base,Rebalanced,Rebalanced_Selected
fit_time,0.000932,0.000913,0.000846
score_time,0.055280,0.030701,0.029522
test_roc_auc,0.902945,0.982638,0.981464
test_matthews_corrcoef,0.633876,0.877475,0.871562
test_f1,0.664559,0.939465,0.936625


Based on every score, rebalanced dataset using SMOTE with every feature, performs the best.

### Hyperparameter tuning

The only hyparameter is number of neighbours

In [73]:
param_grid = {
    'n_neighbors' : np.arange(2, 20, 1)
    }

In [74]:
grid_result = GridSearchCV(KNN, param_grid, scoring = "f1", cv = cv).fit(scal_RE_data_train.values,RE_y_train.values.ravel())

In [75]:
print(grid_result.best_score_,grid_result.best_params_)

0.9607498675783944 {'n_neighbors': 2}


Best number of neighbours turns out to be 2.

In [76]:
KNN = KNeighborsClassifier(n_neighbors=2)
KNN.fit(scal_RE_data_train,RE_y_train)

/Users/anfrejter/Library/Python/3.9/lib/python/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=2)

In [77]:
dump(KNN, '../models/KNeighborsClassifier.joblib') 

['../models/KNeighborsClassifier.joblib']

In [78]:
KNN.score(scal_RE_data_train,RE_y_train)

0.997131931166348

Additionally now we have to normalize our test dataset for further analysis

In [79]:
scaler = StandardScaler()
scal_RE_data_train = pd.DataFrame(scaler.fit_transform(RE_data_train),columns=FE_data_train.columns)


In [80]:
RE_data_test = pd.read_csv('../input_data/fe_X_test.csv',index_col=0)


In [81]:
scal_RE_data_test = pd.DataFrame(scaler.transform(RE_data_test),columns=RE_data_test.columns)

In [82]:
# scal_RE_data_test.to_csv('../input_data/scal_fe_X_test.csv', mode = "w")
# scal_RE_data_train.to_csv('../input_data/scal_re_X_train.csv', mode = "w")

## SVC

In [98]:
clf = SVC()

we will be using data preprocessed for KNN model

In [99]:
res = []
for i in range(len(scal_data)):
    res.append(cross_validate(clf, scal_data[i][0].values, scal_data[i][1].values.ravel(), scoring = scoring ,cv=cv))

### Result

In [100]:
pd.DataFrame([pd.DataFrame(res[0]).mean(),pd.DataFrame(res[1]).mean(),pd.DataFrame(res[2]).mean()]).T.rename(columns={0:'Base',1:'Rebalanced',2:'Rebalanced_Selected'})

,Base,Rebalanced,Rebalanced_Selected
fit_time,0.211903,0.631772,0.628316
score_time,0.107541,0.282823,0.287801
test_roc_auc,0.965364,0.990538,0.990212
test_matthews_corrcoef,0.738287,0.917102,0.912792
test_f1,0.769212,0.958910,0.956743


Based on every score, rebalanced dataset using SMOTE with every feature, performs the best

### Hyperparametru tuning

In [86]:
param_grid = {
    'C': [0.1, 1, 10, 100],  
    'gamma': [1, 0.1, 0.01, 0.001], 
    'kernel': ['linear', 'poly','rbf']}

In [87]:
grid_result = GridSearchCV(clf, param_grid, scoring = "f1", cv = cv, n_jobs=-1).fit(scal_RE_data_train.values,RE_y_train.values.ravel())

/Users/anfrejter/Library/Python/3.9/lib/python/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [89]:
print(grid_result.best_score_,grid_result.best_params_)

0.9780597541931093 {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}


In [123]:
clf = SVC(C = 10, gamma = 0.1, kernel = 'rbf')

In [124]:
clf.fit(scal_RE_data_train.values,RE_y_train.values.ravel())

SVC(C=10, gamma=0.1)

In [125]:
dump(clf, '../models/SupportVectorClassification.joblib') 

['../models/SupportVectorClassification.joblib']

## Random Forest

In [157]:
rf = RandomForestClassifier(random_state=2024)

In [158]:
res = []
for i in range(len(data)):
    res.append(cross_validate(rf, data[i][0].values, data[i][1].values.ravel(), scoring = scoring ,cv=cv))

### Result

In [160]:
pd.DataFrame([pd.DataFrame(res[0]).mean(),pd.DataFrame(res[1]).mean(),pd.DataFrame(res[2]).mean()]).T.rename(columns={0:'Base',1:'Rebalanced',2:'Rebalanced_Selected'})

,Base,Rebalanced,Rebalanced_Selected
fit_time,0.702500,1.547050,1.186531
score_time,0.015306,0.024775,0.026591
test_roc_auc,0.989804,0.998290,0.998192
test_matthews_corrcoef,0.871096,0.961710,0.963296
test_f1,0.890003,0.980929,0.981687


Based on every score, rebalanced dataset using SMOTE with every selected features, performs the best

### Hyperparametru tuning

In [162]:
param_grid = { 
    'n_estimators': [20, 40, 60, 100], 
    'max_features': ['sqrt', 'log2', None], 
    'max_depth': [None ,2, 6, 8], 
    'max_leaf_nodes': [None, 3, 6, 9], 
} 

In [163]:
grid_result = GridSearchCV(rf, param_grid, scoring = "f1", cv = cv, n_jobs=-1).fit(RE_data_train[fe_columns].values,RE_y_train.values.ravel())

In [166]:
print(grid_result.best_score_,grid_result.best_params_)

0.9816874780322139 {'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'n_estimators': 100}


In [167]:
rf = RandomForestClassifier(random_state=2024, max_depth = None, max_features = 'sqrt', max_leaf_nodes = None, n_estimators = 100)

In [170]:
rf.fit(RE_data_train[fe_columns].values,RE_y_train.values.ravel())

RandomForestClassifier(random_state=2024)

In [171]:
dump(rf, '../models/RandomForestClassifier.joblib') 

['../models/RandomForestClassifier.joblib']